# Organize Games from json files

In [9]:
import json
import numpy as np
from collections import OrderedDict
from pathlib import Path

In [10]:
root = Path().absolute().parent
root

WindowsPath('c:/Users/etien/PyProjects/ai2/chris')

## Sort games alphabetically

In [12]:
def load_and_sort(p2json):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
    names = np.array([g['name'] for g in games.values()])
    sorted_game_idxs = np.argsort(names)
    sorted_game_keys = np.array(list(games.keys()))[sorted_game_idxs]
    sorted_list_game_dict = [games[k] for k in sorted_game_keys]
    
    return games, names, sorted_game_idxs, sorted_game_keys, sorted_list_game_dict

def sort_json_by_game_names(p2json):
    odict = OrderedDict()
    games, names, sgi, sgk, slgd = load_and_sort(p2json)
    for i, d in enumerate(slgd):
        odict[str(i+1)] = d

    print(f"{i+1} games")

    with open(p2json, 'w') as fp:
        json.dump(odict, fp, indent=4)

## Search all json files in `config` folder

In [14]:
p2config = root / 'config'
assert p2config.exists()

config_dirs = [p for p in p2config.iterdir() if p.is_dir()]
print(f"There are {len(config_dirs)} config directories")
json_files = [p for p in p2config.glob('**/*.json') if p.suffix == '.json']
for p in config_dirs:
    files = [f for f in p.glob('*.json') if f.is_file()]
    json_files.extend(files)
print(f"There are {len(json_files)} json files in total")

There are 4 config directories
There are 21 json files in total


Sort all json files

In [15]:
for p in json_files:
    print(f"Sorting {p.absolute()}")
    sort_json_by_game_names(p)

Sorting c:\Users\etien\PyProjects\ai2\chris\config\games.json
16 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-07.json
24 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-08.json
21 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-09.json
26 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all.json
25 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games.json
23 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-01-20.json
18 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-05-18-z15.json
21 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-08-31.json
21 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\jam\games.json
20 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\zmack\games.json
20 games
Sorting c:\Users\etien\PyProjects\ai2\chris\config\ai

List all games included in all json files

## Add fields in json

In [33]:
def add_field(p2json, field, save_json=False):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
        for k, v in games.items():
            game_dict = v
            game_dict[field] = game_dict.get(field, "")
            games[k] = game_dict
    
    if save_json:
        with open(p2json, 'w') as fp:
            json.dump(games, fp, indent=4)

    return games

def remove_field(p2json, field, save_json=False):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
        for k, v in games.items():
            game_dict = v
            if field in game_dict.keys():
                del game_dict[field]
                games[k] = game_dict
    
    if save_json:
        with open(p2json, 'w') as fp:
            json.dump(games, fp, indent=4)

    return games

In [10]:
for p2json in json_files:
    # add_field(p2json, 'category', save_json=True);
    add_field(p2json, 'category', save_json=False);

In [79]:
p2refjson = root / 'config/ai2/ai2-games-all-2024-09.json'
with open(p2refjson, 'r') as fp:
    ref_games = json.load(fp)
    
columns = ['jsonfile', 'key'] + list(ref_games['1'].keys())
columns

['jsonfile',
 'key',
 'name',
 'nbr_players',
 'nbr_audience',
 'prompt',
 'exclude',
 'host_include',
 'host_exclude',
 'description',
 'tips',
 'category']

In [82]:
import pandas as pd

df = pd.DataFrame(columns=columns)
for i, p in enumerate(json_files):
    print(f"{i:02d}: Handling {p.absolute()}")
    with open(p, 'r') as fp:
        games = json.load(fp)
    for k, v in games.items():
        cols = ['jsonfile', 'key'] + list(v.keys())
        # values = [str(p.absolute()), k] + list(v.values())
        values = [i, k] + list(v.values())
        row = pd.DataFrame([values], columns=cols, index=[0])
        df = pd.concat([df, row], ignore_index=True)
        
df


00: Handling c:\Users\etien\PyProjects\ai2\chris\config\games.json
01: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-07.json
02: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-08.json
03: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-09.json
04: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all.json
05: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games.json
06: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-01-20.json
07: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-05-18-z15.json
08: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2-show\ai2-games-2024-08-31.json
09: Handling c:\Users\etien\PyProjects\ai2\chris\config\jam\games.json
10: Handling c:\Users\etien\PyProjects\ai2\chris\config\zmack\games.json
11: Handling c:\Users\etien\PyProjects\ai2\chris\config\ai2\ai2-games-all-2024-07.json
12: Handli

,jsonfile,key,name,nbr_players,nbr_audience,prompt,exclude,host_include,host_exclude,description,tips,category
0,0,1,A Date with Me,0,0,A date with me is like <object/occupation>.,[],[],[],None,None,
1,0,2,Double Reverse Alphabet,2,0,A B C D E F G H I J K L M N O P Q R S T U V W ...,[],[],[],None,None,
2,0,3,Forward Reverse,3,0,None,[],[],[],None,None,
3,0,4,Genres,3,0,At least three story genres. \n+ Title of a ne...,[],[],[],None,None,
4,0,5,More or Less,3,0,An ordinary task you do everyday.,[],[],[],None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...
449,20,16,"Story, Story, Die!",0,0,None,[],[Knick],[],None,None,
450,20,17,Stunt Doubles,4,0,None,[],[],[],None,None,
451,20,18,Sure Ding,3,0,None,[],[],[],None,None,
452,20,19,TV Box,4,0,What are four of your favorite TV channels?,[],[],[],None,None,


In [87]:
coi = ['name', 'prompt', 'category', 'jsonfile']
df[coi].sort_values(by='name', ascending=True)

,name,prompt,category,jsonfile
0,A Date with Me,A date with me is like <object/occupation>.,,0
87,A Date with Me,A date with me is like ....,,4
372,A Date with Me,A date with me is like ....,All Play,17
61,A Date with Me,A date with me is like ....,All Play,3
354,A Date with Me,A date with me is like <object/occupation/anim...,NaN,16
...,...,...,...,...
152,World's Worst,"An occupation, animal, or celebrity",NaN,6
234,World's Worst,"An occupation, animal, or celebrity",,10
111,World's Worst,"An occupation, animal, or celebrity",,4
279,World's Worst,"An occupation, animal, or celebrity.",All Play,12


In [86]:
json_files[13]

WindowsPath('c:/Users/etien/PyProjects/ai2/chris/config/ai2/ai2-games-all-2024-09.json')